# Bayesian optimization with fixed features
In some contexts a variable/feature needs to be fixed during optimization. However, we
can leverage previous measurements near the fixed variable value to potentially
jump-start optimization using observed model covariances established by the GP kernel
. In this example, we start with a number of random observations in 2D input space
and then proceed with BO at a fixed value for one of the variables. This notebook
uses the 2D Rosenbrock test function as an example.

In [1]:
# set values if testing
import os
SMOKE_TEST = os.environ.get("SMOKE_TEST")
NUM_MC_SAMPLES = 1 if SMOKE_TEST else 128
NUM_RESTARTS = 1 if SMOKE_TEST else 20

# Ignore all warnings
import warnings
warnings.filterwarnings("ignore")

from xopt import Xopt, Evaluator
from xopt.generators.bayesian import UpperConfidenceBoundGenerator
from xopt.resources.test_functions.rosenbrock import evaluate_rosenbrock,make_rosenbrock_vocs

# make rosenbrock function vocs in 2D
vocs = make_rosenbrock_vocs(2)

# define a fixed value for the BO generator
fixed_features = {"x0":-1.0}
generator = UpperConfidenceBoundGenerator(
    vocs=vocs, fixed_features=fixed_features
)
generator.numerical_optimizer.n_restarts = NUM_RESTARTS
generator.n_monte_carlo_samples = NUM_MC_SAMPLES

evaluator = Evaluator(function=evaluate_rosenbrock)

X = Xopt(generator=generator, evaluator=evaluator, vocs=vocs)
X


            Xopt
________________________________
Version: 2.0a1+20.ge98dd88.dirty
Data size: 0
Config as YAML:
xopt: {asynch: false, strict: true, dump_file: null, max_evaluations: null}
generator:
  name: upper_confidence_bound
  model: null
  turbo_controller: null
  use_cuda: false
  model_constructor:
    name: standard
    use_low_noise_prior: true
    covar_modules: {}
    mean_modules: {}
    trainable_mean_keys: []
  numerical_optimizer: {name: LBFGS, n_raw_samples: 20, n_restarts: 20, max_iter: 2000}
  max_travel_distances: null
  fixed_features: {x0: -1.0}
  n_monte_carlo_samples: 128
  beta: 2.0
evaluator:
  function: xopt.resources.test_functions.rosenbrock.evaluate_rosenbrock
  max_workers: 1
  function_kwargs: {label: y, dummy: 1}
  vectorized: false
vocs:
  variables:
    x0: [-2.0, 2.0]
    x1: [-2.0, 2.0]
  constraints: {}
  objectives: {y: MINIMIZE}
  constants: {}
  observables: []


## Generate some initial random samples in 2D space

In [2]:
X.random_evaluate(3)

,x0,x1,y,xopt_runtime,xopt_error
1,0.161388,-1.485880,229.295377,0.000009,False
2,-1.606027,-1.549724,1711.695483,0.000004,False
3,-0.316046,-1.945083,419.921499,0.000003,False


## Run BO steps with fixed features

In [3]:
for i in range(5):
    X.step()

In [4]:
X.data

,x0,x1,y,xopt_runtime,xopt_error
1,0.161388,-1.485880,229.295377,0.000009,False
2,-1.606027,-1.549724,1711.695483,0.000004,False
3,-0.316046,-1.945083,419.921499,0.000003,False
4,-1.000000,2.000000,104.000000,0.000008,False
5,-1.000000,0.845652,6.382343,0.000009,False
6,-1.000000,1.361574,17.073558,0.000009,False
7,-1.000000,1.063771,4.406678,0.000009,False
8,-1.000000,1.034545,4.119337,0.000009,False
